## Import modules

In [25]:
// set your user name as a case, pointing to your path to documents and tinyir.jar
val (doc_dir: String, tiny_path: String) = System.getProperties().get("user.name").toString match {
    case "Yarden-"  => ("../documents", "../tinyir-1.1.jar")
    case "Max"  => ("../MAXPATH", "../MAXPATH/tinyir-1.1.jar")
}

doc_dir: String = "../documents"
tiny_path: String = "../tinyir-1.1.jar"

In [26]:
classpath.addPath(tiny_path)

In [27]:
import scala.xml.XML
import ch.ethz.dal.tinyir._
import com.github.aztek.porterstemmer.PorterStemmer

import scala.xml.XML
import ch.ethz.dal.tinyir._
import com.github.aztek.porterstemmer.PorterStemmer

In [28]:
// import scala.io.Source  // for importing txt files
import java.io._  // for saving txt files
// import scala.collection.mutable.HashMap  //HashMap used for counting elements in linear time

import java.io._

In [29]:
// import scala.util.Random
import scala.collection.mutable.{Map => MutMap}
// enables "mutable lists"
import scala.collection.mutable.ListBuffer  

import scala.collection.mutable.{Map => MutMap}
import scala.collection.mutable.ListBuffer

## Define classes and functions

In [30]:
def token_filter(text_body: String) = {
    processing.StopWords.filterOutSW(
        processing.Tokenizer.tokenize(text_body.
                                      replaceAll("\\P{L}+", " "))
    ).
    map(x => PorterStemmer.stem(x)).filter(_.trim.nonEmpty)
}

defined function token_filter

In [31]:
class xml_doc (file_path: String) {
    def get_doc(): xml.Elem = {
        XML.loadFile(file_path: String)
    }    
    
    def text() = {
        (get_doc() \\ "DOC" \\ "TEXT").text
    }
    
    def head() = {
        (get_doc() \\ "DOC" \\ "HEAD").text
    }

    def id() = {
        (get_doc() \\ "DOC" \\ "DOCNO").text.trim
    }
    
    def tokens() = {
        token_filter(head() ++ text())
    }
}

defined class xml_doc

In [32]:
def list_docs (path: String) = {  // : Array[java.io.File]
        new java.io.File(path).listFiles.map(x => x.toString())
    }
val numPattern = "[0-9]+".r

defined function list_docs
numPattern: scala.util.matching.Regex = [0-9]+

In [33]:
def write(data: MutMap[Double, (Double, Double, Double)],filename: String) = {

    val bw = new BufferedWriter(new FileWriter(new File(filename)))
    val iter = data.keys.iterator
    while(iter.hasNext){
        var elem = iter.next()
        var values = data(elem).productIterator.toList
        if(values.length>0){
            bw.write(elem+" "+values.mkString(" "))
            bw.newLine
        }    
    }   
    bw.close()
}

def write_prediction(data: MutMap[Int, Set[String]],filename: String) = {

    val bw = new BufferedWriter(new FileWriter(new File(filename)))
    val iter = data.keys.iterator
    while(iter.hasNext){
        var elem = iter.next()
        var values = data(elem).toList
        if(values.length>0){
            bw.write(elem+" "+values.mkString(" "))
            bw.newLine
        }    
    }   
    bw.close()
}

defined function write
defined function write_prediction

# Importing data files

In [34]:
val train_list = list_docs(doc_dir)

train_list: Array[String] = Array(
  """
..\documents\AP880212-0006
  """,
  """
..\documents\AP880212-0007
  """,
  """
..\documents\AP880212-0009
  """,
  """
..\documents\AP880212-0017
  """,
  """
..\documents\AP880212-0018
  """,
  """
..\documents\AP880212-0022
  """,
  """
...

In [ ]:
val doc_list = new ListBuffer[processing.StringDocument]()
var counter = 0
// for (path <- train_list.take(1000)){
for (path <- train_list){
    var cur_doc = new xml_doc(path)
    doc_list += (new processing.StringDocument(counter, cur_doc.tokens.mkString(" ")))
    counter += 1
    if (counter % 50 == 0) println(s"iteration $counter")
}

In [ ]:
val doc_stream = doc_list.toStream

In [ ]:
val test_PosIndex = new indexing.PosIndex(doc_stream)

In [ ]:
test_PosIndex.index
// test_pos_index.postings(doc_stream)

In [ ]:
test_PosIndex.results("iowa")

In [ ]:
test_PosIndex.results("coupl")

In [ ]:
indexing.ProxWindow.size

In [ ]:
test_PosIndex.results(Seq("iowa", "coupl"))

In [ ]:
indexing.ProxWindow.setSize(2)

In [ ]:
test_PosIndex.results(Seq("iowa", "withkemp"))

In [ ]:
val test_SimpleIndex = new indexing.SimpleIndex(doc_stream)

In [ ]:
test_SimpleIndex.index
test_SimpleIndex.index.mapValues(x => x.length)

In [ ]:
val test_FreqIndex = new indexing.FreqIndex(doc_stream)

In [ ]:
test_FreqIndex.index
test_FreqIndex.results("iowa")

In [ ]:
val test_InvertedIndex = indexing.InvertedIndex

#### creating list of files
#### indexing => creating inverse index
#### lectures(?) => look at scoring algorithms

# Testing ground

In [ ]:
val mb = 1024*1024
val runtime = Runtime.getRuntime
println(s"Used Memory:  " + (runtime.totalMemory - runtime.freeMemory) / mb)
println(s"Free Memory:  " + runtime.freeMemory / mb)
println(s"Total Memory: " + runtime.totalMemory / mb)
println(s"Max Memory:   " + runtime.maxMemory / mb)